In [5]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TreebankWordTokenizer
import spacy
from nltk.corpus import stopwords

In [6]:
train=pd.read_csv('train.csv')

In [7]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
train.shape

(7613, 5)

In [9]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [10]:
train.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [11]:
train['keyword'].unique()

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [12]:
train.target.unique()

array([1, 0], dtype=int64)

In [13]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [14]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.countplot(train.target)

In C:\Users\Lenovo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\Lenovo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Lenovo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\Lenovo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Lenovo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


# Clean text

In [15]:
import re
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(document):
    return TAG_RE.sub('', document)


def clean(doc):
    doc=doc.lower()
    document = remove_tags(doc)
    document = re.sub('[^a-zA-Z]', ' ', document)
    document = re.sub(r"\s+[a-zA-Z]\s+", ' ', document)
    document = re.sub(r'\s+', ' ', document)
    
    return document

In [16]:
train['clean_text']=train['text'].apply(clean)

In [17]:
train['clean_text'][0]

'our deeds are the reason of this earthquake may allah forgive us all'

In [18]:
train.head()

,id,keyword,location,text,target,clean_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [19]:
train['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [20]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [21]:
train[train["target"] == 1]["clean_text"].values[1]

'forest fire near la ronge sask canada'

In [22]:
train[train['target']==0]["clean_text"].values[0]

'what up man '

In [23]:
train[train['target']==1]["clean_text"].values[0]

'our deeds are the reason of this earthquake may allah forgive us all'

##### Count the number words in each tweet

In [24]:
count_vector=feature_extraction.text.CountVectorizer()
# from the first 10 tweets
train_d=count_vector.fit_transform(train["clean_text"][0:10])

In [25]:
print('There are ',train_d[0].todense().shape[1] ,'unique words in the first 10 words')

There are  95 unique words in the first 10 words


In [26]:
print(train_d.todense())

[[0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
  0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 2 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0
  0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 1 0 0 0 2 0 0 0 0 1
  0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 2 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0
  0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0

In [27]:
train_df=count_vector.fit_transform(train["clean_text"])

In [28]:
print('There are ',train_df.todense().shape[1],'unique words')

There are  22101 unique words


#### Model

In [29]:
from sklearn import linear_model
clf=linear_model.RidgeClassifier()

In [30]:
scores=model_selection.cross_val_score(clf,train_df,train['target'],cv=3,scoring='f1')

In [31]:
scores

array([0.6149761 , 0.55566406, 0.63178677])

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
score=model_selection.cross_val_score(rf,train_df,train['target'])

In [33]:
score

array([0.72882469, 0.66053841, 0.68286277, 0.69842313, 0.75886991])

In [34]:
from sklearn.linear_model import SGDClassifier
sgd=SGDClassifier()
scored=model_selection.cross_val_score(sgd,train_df,train['target'])

In [35]:
scored

array([0.7202889 , 0.63755745, 0.66185161, 0.66885677, 0.73587385])

In [36]:
rf.fit(train_df,train['target'])

RandomForestClassifier()

In [37]:
test=pd.read_csv('test.csv')
test_v=count_vector.transform(test['text'])

In [51]:
# sample_submission = pd.read_csv("sample_submission.csv")
# sample_submission["target"] = rf.predict(test_v)
# sample_submission.head()


,id,target
0,0,1
1,2,1
2,3,1
3,9,0
4,11,1


In [54]:
# sample_submission.to_csv('sub1.csv',index=None)

In [38]:
from sklearn.linear_model import LogisticRegression
lg=logreg = LogisticRegression(n_jobs=1, C=1e5)

In [39]:
scorel=model_selection.cross_val_score(lg,train_df,train['target'])

C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-r

In [40]:
scorel

array([0.70781353, 0.61457649, 0.65200263, 0.65308804, 0.71879106])

In [41]:
from sklearn.naive_bayes import MultinomialNB
nb=MultinomialNB()
scor=model_selection.cross_val_score(nb,train_df,train['target'])

In [42]:
scor

array([0.71240972, 0.6684176 , 0.7196323 , 0.70827858, 0.75492773])

In [43]:
nb.fit(train_df,train['target'])

MultinomialNB()

In [44]:
# sample_submission = pd.read_csv("sample_submission.csv")
# sample_submission["target"] = nb.predict(test_v)
# sample_submission.head()

In [45]:
# sample_submission.to_csv('sub2.csv',index=None)

# KN CLASSIFIER

In [55]:
from sklearn.neighbors import KNeighborsClassifier
kn=KNeighborsClassifier()
scorel=model_selection.cross_val_score(kn,train_df,train['target'])

In [56]:
kn.fit(train_df,train['target'])
sample_submission["target"] = kn.predict(test_v)
sample_submission.to_csv('sub7.csv',index=None)

In [57]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,0
3,9,0
4,11,0


In [46]:
from sklearn.ensemble import StackingClassifier
multi_layer = StackingClassifier(estimators=[('KNclassifier', KNeighborsClassifier()),
                                                      ('multi', MultinomialNB()),('sgd', SGDClassifier()),('random_forest',RandomForestClassifier())],
                                                                                                 final_estimator=clf)
multi_layer.fit(train_df,train['target'])

StackingClassifier(estimators=[('KNclassifier', KNeighborsClassifier()),
                               ('multi', MultinomialNB()),
                               ('sgd', SGDClassifier()),
                               ('random_forest', RandomForestClassifier())],
                   final_estimator=RidgeClassifier())

In [47]:
score_s=model_selection.cross_val_score(multi_layer,train_df,train['target'])

In [48]:
score_s

array([0.74130007, 0.69139856, 0.73539068, 0.72141919, 0.7890933 ])

In [50]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission["target"] = multi_layer.predict(test_v)
sample_submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [51]:
sample_submission.to_csv('sub6.csv',index=None)

In [81]:
# sample_submission.to_csv('sub3.csv',index=None)

In [52]:
vectorizer = feature_extraction.text.TfidfVectorizer(use_idf=True, stop_words = 'english')

In [53]:
train_data=vectorizer.fit_transform(train['clean_text'])

In [54]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
multi_layer.fit(train_data,train['target'])
score_s1=model_selection.cross_val_score(multi_layer,train_data,train['target'])

In [87]:
score_s1

array([0.7242285 , 0.67301379, 0.71437951, 0.71419185, 0.76346912])

In [ ]:
sample_submission["target"] = multi_layer.predict(test_v)
sample_submission.head()

In [ ]:
sample_submission.to_csv('sub6.csv',index=None)